# Homework 8: Deep Learning with PyTorch

In this homework, we'll build a model for classifying various hair types.

In [1]:
import numpy as np
import torch
import torch.nn as nn
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)

if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    device = torch.device('cuda')
elif torch.backends.mps.is_available():
    device = torch.device('mps')
else:
    device = torch.device('cpu')
print(f"Using device: {device}")

PyTorch version: 2.8.0
Using device: mps


## Data Preparation

In [2]:
train_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

test_transforms = transforms.Compose([
    transforms.Resize((200, 200)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset = ImageFolder('./data/train', transform=train_transforms)
test_dataset = ImageFolder('./data/test', transform=test_transforms)

train_loader = DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=0)
test_loader = DataLoader(test_dataset, batch_size=20, shuffle=False, num_workers=0)

classes = train_dataset.classes
print(f"Classes: {classes}")

Classes: ['curly', 'straight']


## Model Definition

In [3]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2)
        # Input: 200x200
        # Conv (3x3): 198x198
        # Pool (2x2): 99x99
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(32 * 99 * 99, 64)
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = self.conv1(x)
        x = self.relu(x)
        x = self.pool(x)
        x = self.flatten(x)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

model = CNN().to(device)

### Question 1: Loss Function

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.002, momentum=0.8)
criterion = nn.BCEWithLogitsLoss()
print("Loss function: BCEWithLogitsLoss")

Loss function: BCEWithLogitsLoss


### Question 2: Number of Parameters

In [5]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 20073473


## Training

In [6]:
num_epochs = 10
history = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset)
    epoch_acc = correct_train / total_train
    history['loss'].append(epoch_loss)
    history['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history['val_loss'].append(val_epoch_loss)
    history['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.8395, Acc: 0.5875, Val Loss: 0.5452, Val Acc: 0.7313
Epoch 2/10, Loss: 0.6486, Acc: 0.6525, Val Loss: 0.6809, Val Acc: 0.6219
Epoch 3/10, Loss: 0.5635, Acc: 0.6750, Val Loss: 0.5867, Val Acc: 0.6816
Epoch 4/10, Loss: 0.5102, Acc: 0.6987, Val Loss: 0.6818, Val Acc: 0.6517
Epoch 5/10, Loss: 0.5199, Acc: 0.7200, Val Loss: 0.6493, Val Acc: 0.6716
Epoch 6/10, Loss: 0.4586, Acc: 0.7688, Val Loss: 0.6365, Val Acc: 0.6816
Epoch 7/10, Loss: 0.4038, Acc: 0.7963, Val Loss: 0.7079, Val Acc: 0.6517
Epoch 8/10, Loss: 0.3540, Acc: 0.8387, Val Loss: 0.8420, Val Acc: 0.6368
Epoch 9/10, Loss: 0.3583, Acc: 0.8200, Val Loss: 0.6204, Val Acc: 0.7264
Epoch 10/10, Loss: 0.2796, Acc: 0.8575, Val Loss: 0.6796, Val Acc: 0.6816


### Question 3: Median Training Accuracy

In [7]:
median_acc = np.median(history['acc'])
print(f"Median training accuracy: {median_acc}")

Median training accuracy: 0.744375


### Question 4: Standard Deviation of Training Loss

In [8]:
std_loss = np.std(history['loss'])
print(f"Standard deviation of training loss: {std_loss}")

Standard deviation of training loss: 0.15568790421534814


## Data Augmentation

In [9]:
train_transforms_aug = transforms.Compose([
    transforms.RandomRotation(50),
    transforms.RandomResizedCrop(200, scale=(0.9, 1.0), ratio=(0.9, 1.1)),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

train_dataset_aug = ImageFolder('./data/train', transform=train_transforms_aug)
train_loader_aug = DataLoader(train_dataset_aug, batch_size=20, shuffle=True, num_workers=0)

### Continued Training

In [10]:
num_epochs_aug = 10
history_aug = {'acc': [], 'loss': [], 'val_acc': [], 'val_loss': []}

for epoch in range(num_epochs_aug):
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0
    for images, labels in train_loader_aug:
        images, labels = images.to(device), labels.to(device)
        labels = labels.float().unsqueeze(1)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * images.size(0)
        predicted = (torch.sigmoid(outputs) > 0.5).float()
        total_train += labels.size(0)
        correct_train += (predicted == labels).sum().item()

    epoch_loss = running_loss / len(train_dataset_aug)
    epoch_acc = correct_train / total_train
    history_aug['loss'].append(epoch_loss)
    history_aug['acc'].append(epoch_acc)

    model.eval()
    val_running_loss = 0.0
    correct_val = 0
    total_val = 0
    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            labels = labels.float().unsqueeze(1)

            outputs = model(images)
            loss = criterion(outputs, labels)

            val_running_loss += loss.item() * images.size(0)
            predicted = (torch.sigmoid(outputs) > 0.5).float()
            total_val += labels.size(0)
            correct_val += (predicted == labels).sum().item()

    val_epoch_loss = val_running_loss / len(test_dataset)
    val_epoch_acc = correct_val / total_val
    history_aug['val_loss'].append(val_epoch_loss)
    history_aug['val_acc'].append(val_epoch_acc)

    print(f"Epoch {epoch+1}/{num_epochs_aug}, "
          f"Loss: {epoch_loss:.4f}, Acc: {epoch_acc:.4f}, "
          f"Val Loss: {val_epoch_loss:.4f}, Val Acc: {val_epoch_acc:.4f}")

Epoch 1/10, Loss: 0.6490, Acc: 0.6025, Val Loss: 0.6455, Val Acc: 0.6169
Epoch 2/10, Loss: 0.5980, Acc: 0.6138, Val Loss: 0.9842, Val Acc: 0.6169
Epoch 3/10, Loss: 0.6378, Acc: 0.6362, Val Loss: 0.6296, Val Acc: 0.5920
Epoch 4/10, Loss: 0.6202, Acc: 0.6250, Val Loss: 0.6276, Val Acc: 0.6816
Epoch 5/10, Loss: 0.5781, Acc: 0.6963, Val Loss: 0.6460, Val Acc: 0.6219
Epoch 6/10, Loss: 0.5622, Acc: 0.6925, Val Loss: 0.6122, Val Acc: 0.6965
Epoch 7/10, Loss: 0.5560, Acc: 0.7175, Val Loss: 0.6873, Val Acc: 0.6418
Epoch 8/10, Loss: 0.5261, Acc: 0.7238, Val Loss: 0.6902, Val Acc: 0.6368
Epoch 9/10, Loss: 0.5347, Acc: 0.7238, Val Loss: 0.6090, Val Acc: 0.6567
Epoch 10/10, Loss: 0.5264, Acc: 0.7300, Val Loss: 0.6087, Val Acc: 0.7114


### Question 5: Mean Test Loss (Augmented)

In [11]:
mean_val_loss = np.mean(history_aug['val_loss'])
print(f"Mean test loss (augmented): {mean_val_loss}")

Mean test loss (augmented): 0.6740420799350264


### Question 6: Average Test Accuracy (Last 5 Epochs, Augmented)

In [12]:
avg_val_acc_last_5 = np.mean(history_aug['val_acc'][5:])
print(f"Average test accuracy (last 5 epochs, augmented): {avg_val_acc_last_5}")

Average test accuracy (last 5 epochs, augmented): 0.6686567164179105
